In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import xlsxwriter


import warnings
warnings.filterwarnings('ignore')

from flask import Flask
from flask import request
from flask import render_template
import mysql.connector
from mysql.connector import Error
import string

In [16]:
# Connect to MYSQL Database
connection = mysql.connector.connect(host='192.168.12.102',
            database='vonly_data_feed_us_staging',
            user='vonly-agent',
            password='a714fded-311c-4215-8b8b-5df4086e264b')

In [17]:
def token(text):
    # Remove everything except normal alphabets and numbers
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Tokenize the cleaned text
    tokens = cleaned_text.split()
    result =""
    for a in tokens:
        result = result + a
    result = result.lower()
    for i in range(50):
        result = result.replace("pt"+str(i),"part"+str(i))
        result = result.replace("p"+str(i),"part"+str(i))
        result = result.replace("partone","part1")
        result = result.replace("parttwo","part2")
        result = result.replace("partthree","part3")
        result = result.replace("partfour","part4")
        result = result.replace("partfive","part5")
        result = result.replace("volume","vol")
        
    return result

In [18]:
def get_v_id():
    query100 = """ 
                    SELECT DISTINCT(t.vonly_asset_id) 
                    FROM tv_ids t 
                    WHERE t.scope='tv_season' AND
                    t.platform NOT IN ('Amazon Movies & TV','WB MPM-VVId','YouTube','WB UPC')
    """

    df100 = pd.read_sql_query(query100, connection)
    
    return df100

In [19]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

def make_excel(data):
    df = pd.DataFrame(data) 
    x = get_var_name(data)
    df.to_excel("{}.xlsx".format(x))

In [20]:
def get_data(i):
    query0 = """ 
                    SELECT * 
                    FROM tv_ids ti 
                    WHERE 
                    ti.scope='tv_season' 
                    AND ti.vonly_asset_id='{}'
                    AND platform NOT IN ('Amazon Movies & TV','WB UPC','YouTube')
    """.format(i)

    df0 = pd.read_sql_query(query0, connection)
    
    return df0

In [21]:
def get_season_metadata(df0):
    res1 = pd.DataFrame()  # Initialize an empty DataFrame

    for portal_item_id, Platform in zip(df0['portal_item_id'], df0['platform']):
        query1 = """
        SELECT ti.vonly_asset_id,ti.title season_name,ti.portal_item_id season_portal_item_id,ti.num season_no,
                                                t.description,t.distributed_by,t.distributed_by_parent,t.produced_by,t.produced_by_parent,
                                                t.us_theater_release_date,t.director,t.actors,t.writers,t.producers,
                                                t.runtime,t.num season_no2, ti.platform, 
                                                (select count(1) from tv_ids tii where tii.scope='tv_season' and tii.vonly_asset_id=ti.vonly_asset_id and tii.platform=ti.platform) va_count
                                                FROM vonly_data_feed_us_staging.tv_ids ti
                                                INNER JOIN sandbox.tvshows t ON ti.id=t.vonly_id AND ti.region=t.region 
                                                 WHERE
                                                ti.scope='tv_season' AND ti.platform NOT IN ('amazon movies & tv') AND  
                                                ti.portal_item_id='{0}' and ti.platform='{1}' 

        """.format(portal_item_id, Platform)
    
        df1 = pd.read_sql_query(query1, connection)
         
        res1 = pd.concat([res1, df1], ignore_index=True)
        
    return res1

In [22]:
def get_episode_details(df0):
    
    res2 = []
    
    for portal_item_id, Platform in zip(df0['portal_item_id'], df0['platform']):
        query2 = """
        SELECT ti.vonly_asset_id,ti.title season_name,ti.portal_item_id season_portal_item_id,ti.num season_no,
                                                t.description,t.distributed_by,t.distributed_by_parent,t.produced_by,t.produced_by_parent,
                                                t.us_theater_release_date,t.director,t.actors,t.writers,t.producers,
                                                t.runtime,t.num season_no2,tie.title episode_name,tie.portal_item_id episode_portal_item_id, tie.num episode_no, ti.platform, 
                                                (select count(1) from tv_ids tii where tii.scope='tv_season' and tii.vonly_asset_id=ti.vonly_asset_id and tii.platform=ti.platform) va_count
                                                FROM vonly_data_feed_us_staging.tv_ids ti
                                                INNER JOIN sandbox.tvshows t ON ti.id=t.vonly_id AND ti.region=t.region 
                                                LEFT OUTER JOIN vonly_data_feed_us_staging.tv_id_mappings tim ON ti.id=tim.season_vonly_id
                                                LEFT OUTER JOIN  vonly_data_feed_us_staging.tv_ids tie ON tim.episode_vonly_id=tie.id  AND tie.scope='tv_episode'
                                                 WHERE
                                                ti.scope='tv_season' AND ti.platform NOT IN ('Amazon Movies & TV','WB UPC','YouTube') AND 
                                               ti.portal_item_id='{0}' and ti.platform='{1}' 
                                               ORDER BY CAST(tie.num AS SIGNED)

        """.format(portal_item_id,Platform)
              
        df2 = pd.read_sql_query(query2, connection)
        res2.append(df2)
        
    return res2

In [23]:
def metadata_validation(res):
    
    a = res[res['platform']  == 'AppleTVApp']
    i = res[res['platform']  == 'iTunes']
    am = res[res['platform']  == 'Amazon Prime Video']
    v = res[res['platform']  == 'VUDU']
    g = res[res['platform']  == 'Google Play']

    if i.shape[0] != 0:
        master = i.head(1)
    elif v.shape[0] != 0:
        master = v.head(1)
    elif g.shape[0] != 0:
        master = g.head(1)
    elif am.shape[0] != 0:
        master = am.head(1)
    elif a.shape[0] != 0:
        master = a.head(1)
        
    mapping_list = []
    id_ = 0

    for _, row1 in res.iterrows():
        id_ += 1

        count = 0
        mapped = 0

        matching_points = {
            'id': id_,
            'season': 0,
            'season_no':row1['season_no'],
            'season_no_1': '',
            'title': 0,
            'title_name': row1['season_name'],
            'title_name_1': '',
            'director': 0,
            'director_name': row1['director'],
            'director_name_1':'' ,
            'producers': 0,
            'producer_name': row1['producers'],
            'producer_name_1':'' ,
            'portal_item_id': 0,
            'portal_item_id_': row1['season_portal_item_id'],
            'portal_item_id_1':'' ,
            'writers': 0,
            'writer_name': row1['writers'],
            'writer_name_1':'' ,
            'runtime': 0,
            'runtime_': row1['runtime'],
            'runtime_1': '',
            'count': count,
            'mapped': mapped,
        }

        for _, row2 in master.iterrows():
# 1
            if not pd.isnull(row1['season_no']) and not pd.isnull(row2['season_no']) and row1['season_no'] == row2['season_no']:
                matching_points['season'] = 1
            elif pd.isnull(row1['season_no']) and pd.isnull(row2['season_no']):
                matching_points['season'] = 0
            matching_points['season_no_1'] = row2['season_no']

# 2
            if not pd.isnull(row1['season_name']) and not pd.isnull(row2['season_name']):
                m = token(row1['season_name'])
                n = token(row2['season_name'])
                if m >= n and m.find(n) != -1:
                    matching_points['title'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['title'] = 1
            elif pd.isnull(row1['season_name']) and pd.isnull(row2['season_name']):
                matching_points['title'] = 0
            matching_points['title_name_1'] = row2['season_name']

# 3
            if not pd.isnull(row1['director']) and not pd.isnull(row2['director']):
                m = token(row1['director'])
                n = token(row2['director'])
                if m >= n and m.find(n) != -1:
                    matching_points['director'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['director'] = 1
            elif pd.isnull(row1['director']) and pd.isnull(row2['director']):
                matching_points['director'] = 0
            matching_points['director_name_1'] = row2['director']

# 4
            if not pd.isnull(row1['producers']) and not pd.isnull(row2['producers']):
                m = token(row1['producers'])
                n = token(row2['producers'])
                if m >= n and m.find(n) != -1:
                    matching_points['producers'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['producers'] = 1
            elif pd.isnull(row1['producers']) and pd.isnull(row2['producers']):
                matching_points['producers'] = 0
            matching_points['producer_name_1'] = row2['producers']

# 5
            if not pd.isnull(row1['season_portal_item_id']) and not pd.isnull(row2['season_portal_item_id']):
                m = token(row1['season_portal_item_id'])
                n = token(row2['season_portal_item_id'])
                if m >= n and m.find(n) != -1:
                    matching_points['portal_item_id'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['portal_item_id'] = 1
            elif pd.isnull(row1['season_portal_item_id']) and pd.isnull(row2['season_portal_item_id']):
                matching_points['portal_item_id'] = 0
            matching_points['portal_item_id_1'] = row2['season_portal_item_id']

# 6
            if not pd.isnull(row1['writers']) and not pd.isnull(row2['writers']):
                m = token(row1['writers'])
                n = token(row2['writers'])
                if m >= n and m.find(n) != -1:
                    matching_points['writers'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['writers'] = 1
            elif pd.isnull(row1['writers']) and pd.isnull(row2['writers']):
                matching_points['writers'] = 0
            matching_points['writer_name_1'] = row2['writers']

# 7
            if not pd.isnull(row1['runtime']) and not pd.isnull(row2['runtime']):
                m = token(row1['runtime'])
                n = token(row2['runtime'])
                if m >= n and m.find(n) != -1:
                    matching_points['runtime'] = 1
                elif n > m and n.find(m)!= -1:
                    matching_points['runtime'] = 1
            elif pd.isnull(row1['runtime']) and pd.isnull(row2['runtime']):
                matching_points['runtime'] = 0
            matching_points['runtime_1'] = row2['runtime']

            count = matching_points['season'] + matching_points['title'] + matching_points['director'] + matching_points['producers'] + matching_points['portal_item_id'] + matching_points['writers'] + matching_points['runtime']

        mapped = 1 if count >= 3 else 0

        matching_points['count'] = count
        matching_points['mapped'] = mapped

        mapping_list.append(matching_points)

    return mapping_list

In [15]:
# def episode_validation1(res2):
    
#     for x in res2:
#         #print(x['platform'][0])
#         if x['platform'][0] == 'iTunes':
#             master = x
#             break
#         elif x['platform'][0] == 'VUDU':
#             master = x
#             break
#         elif x['platform'][0] == 'Google Play':
#             master = x
#             break
#         elif x['platform'][0] == 'Amazon Prime Video':
#             master = x
#             break
#         elif x['platform'][0] == 'AppleTVApp':
#             master = x
#             break
        
# #     cnt=0

#     mapping_list = []
#     id_ = 0
 
#     for r in res2:  
        
#         count = 0
#         mapped = 0

#         matching_points = {
#             'id': 0,
#             'vonly_asset_id_dest':'',
#             'vonly_asset_id_source':'',
#             'season_no_dest':'',
#             'season_no_source':'',
#             'season_name_dest':'',
#             'season_name_1_source':'',
#             'episode_platform_dest': '',
#             'episode_platform_source':'',
#             'episode_portal_item_id_dest': '',
#             'episode_portal_item_id_source':'',
#             'episodes': 0,
#             'episode_no_dest':'',
#             'episode_no_source': '',
#             'episode_names': 0,
#             'episode_name_dest': '',
#             'episode_name_source': '',
#             'count': 0,
#             'mapped': 0,
#         }

#         for _, row1 in r.iterrows():
            
#             if row1['episode_name'] == None or row1['episode_name'] == '' or row1['episode_name'] == 'None' :
#                         continue
            
#             for _, row2 in master.iterrows():
                
#                 matching_points = {
#                     'id': 0,
                     
#                     'vonly_asset_id_dest':'',
#                     'vonly_asset_id_source':'',
                     
#                     'season_no_dest':'',
#                     'season_no_source':'',
                     
#                     'season_name_dest':'',
#                     'season_name_source':'',
                     
#                     'episode_platform_dest': '',
#                     'episode_platform_source':'',
                     
#                     'episode_portal_item_id_dest': '',
#                     'episode_portal_item_id_source':'',

#                     'episode_no_dest':'',
#                     'episode_no_source': '',
                     
#                     'episode_names': 0,
#                     'episode_name_dest': '',
#                     'episode_name_source': '',
                     
#                     'count': 0,
#                     'mapped': 0,
#                 }
              
# # 1
#                 if row1['episode_no'] == row2['episode_no']:
#                     id_ += 1
#                     matching_points['id'] = id_
                    
# #                     cnt = cnt+1
# #                     print(cnt,row2['episode_name'],row1['episode_name'])
                   
#                     matching_points['episode_no_dest'] = row1['episode_no']
#                     matching_points['episode_no_source'] = row2['episode_no']
#                     print( row2['episode_name'])
#                     if row2['episode_name'] == None or row2['episode_name'] == '' or row2['episode_name'] == 'None' :
#                         continue
#                     v = row2['episode_name'].lower()
#                     plt2= v.find("pilot")
#                     u = row1['episode_name'].lower()
#                     plt= u.find("pilot")

# # 2
#                     if not pd.isnull(row1['episode_name']) and not pd.isnull(row2['episode_name']) and (plt2 == -1 and plt2 == -1):
#                         m = token(row1['episode_name'])
#                         n = token(row2['episode_name'])
#                         if m >= n and m.find(n) != -1:
#                             count+=1
#                         elif n > m and n.find(m)!= -1:
#                             count+=1

#                     matching_points['episode_name_dest'] = row1['episode_name']
#                     matching_points['episode_name_source'] = row2['episode_name']
                    
#                     matching_points['vonly_asset_id_dest'] = row1['vonly_asset_id']
#                     matching_points['vonly_asset_id_source'] = row2['vonly_asset_id']
                    
#                     matching_points['season_no_dest'] = row1['season_no']
#                     matching_points['season_no_source'] = row2['season_no']
                    
#                     matching_points['season_name_dest'] = row1['season_name']
#                     matching_points['season_name_source'] = row2['season_name']
                    
#                     matching_points['episode_platform_dest'] = row1['platform']
#                     matching_points['episode_platform_source'] = row2['platform']
                    
#                     matching_points['episode_portal_item_id_dest'] = row1['episode_portal_item_id']
#                     matching_points['episode_portal_item_id_source'] = row2['episode_portal_item_id']
                    
#                     matching_points['episode_names'] = count

#                     mapped = 1 if count >= 5 else 0

#                     matching_points['count'] = count
#                     matching_points['mapped'] = mapped


#                     mapping_list.append(matching_points)

#     return mapping_list

In [10]:
# import pandas as pd

# def episode_validation1(res2):
#     mapping_list = []
#     id_ = 0
#     platform_found = False

#     for r in res2:
#         if isinstance(r, pd.DataFrame) and 'platform' in r.columns:  
#             platforms = r['platform'].tolist()  
#             if platforms:  
#                 platform = platforms[0]  
#                 if platform in ['iTunes', 'VUDU', 'Google Play', 'Amazon Prime Video', 'AppleTVApp']:
#                     master = r
#                     platform_found = True
#                     break
#                 if pd.isnull(r['platform']) or r['platform'] == '':
#                     continue
#         elif isinstance(r, pd.Series) and 'platform' in r.index:  
#             platform = r['platform']  
#             if platform in ['iTunes', 'VUDU', 'Google Play', 'Amazon Prime Video', 'AppleTVApp']:
#                 master = pd.DataFrame(r).T  
#                 platform_found = True
#                 break
#             if pd.isnull(r['platform']) or r['platform'] == '':
#                 continue

#     if not platform_found:
#         print("No valid platform found in res2")

#     for r in res2:
#         count = 0
#         mapped = 0

#         for _, row1 in r.iterrows():
#             if pd.isnull(row1['episode_name']) or row1['episode_name'] == '':
#                 continue

#             for _, row2 in master.iterrows():
#                 matching_points = {
#                     'id': id_,
#                     'vonly_asset_id_dest': row1['vonly_asset_id'],
#                     'vonly_asset_id_source': row2['vonly_asset_id'],
#                     'season_no_dest': row1['season_no'],
#                     'season_no_source': row2['season_no'],
#                     'season_name_dest': row1['season_name'],
#                     'season_name_source': row2['season_name'],
#                     'episode_platform_dest': row1['platform'],
#                     'episode_platform_source': row2['platform'],
#                     'episode_portal_item_id_dest': row1['episode_portal_item_id'],
#                     'episode_portal_item_id_source': row2['episode_portal_item_id'],
#                     'episode_no_dest': row1['episode_no'],
#                     'episode_no_source': row2['episode_no'],
#                     'episode_names': 0,
#                     'episode_name_dest': row1['episode_name'],
#                     'episode_name_source': row2['episode_name'],
#                     'count': 0,
#                     'mapped': 0,
#                 }

#                 if row1['episode_no'] == row2['episode_no']:
#                     id_ += 1
                    
#                     matching_points['id'] = id_
                    
# #                     cnt = cnt+1
# #                     print(cnt,row2['episode_name'],row1['episode_name'])
                   
#                     matching_points['episode_no_dest'] = row1['episode_no']
#                     matching_points['episode_no_source'] = row2['episode_no']


#                     if not pd.isnull(row2['episode_name']):
#                         v = row2['episode_name'].lower()
#                         plt2 = v.find("pilot")
#                         u = row1['episode_name'].lower()
#                         plt = u.find("pilot")

#                         if plt2 == -1 and plt2 == -1:
#                             m = token(row1['episode_name'])
#                             n = token(row2['episode_name'])
#                             if m >= n and m.find(n) != -1:
#                                 count += 1
#                             elif n > m and n.find(m) != -1:
#                                 count += 1
                                
                    
#                     matching_points['episode_name_dest'] = row1['episode_name']
#                     matching_points['episode_name_source'] = row2['episode_name']
                    
#                     matching_points['vonly_asset_id_dest'] = row1['vonly_asset_id']
#                     matching_points['vonly_asset_id_source'] = row2['vonly_asset_id']
                    
#                     matching_points['season_no_dest'] = row1['season_no']
#                     matching_points['season_no_source'] = row2['season_no']
                    
#                     matching_points['season_name_dest'] = row1['season_name']
#                     matching_points['season_name_source'] = row2['season_name']
                    
#                     matching_points['episode_platform_dest'] = row1['platform']
#                     matching_points['episode_platform_source'] = row2['platform']
                    
#                     matching_points['episode_portal_item_id_dest'] = row1['episode_portal_item_id']
#                     matching_points['episode_portal_item_id_source'] = row2['episode_portal_item_id']
                    
#                     matching_points['episode_names'] = count

#                     mapped = 1 if count >= 5 else 0

#                     matching_points['count'] = count
#                     matching_points['mapped'] = mapped


#                     mapping_list.append(matching_points)


#     return mapping_list


In [24]:
import pandas as pd

def episode_validation1(res2):
    mapping_list = []
    id_ = 0
    platform_found = False

    for r in res2:
        if isinstance(r, pd.DataFrame) and 'platform' in r.columns:  
            platforms = r['platform'].tolist()  
            if platforms:  
                platform = platforms[0]  
                if platform in ['iTunes', 'VUDU', 'Google Play', 'Amazon Prime Video', 'AppleTVApp']:
                    master = r
                    platform_found = True
                    break
                if pd.isnull(r['platform']) or r['platform'] == '':
                    continue
        elif isinstance(r, pd.Series) and 'platform' in r.index:  
            platform = r['platform']  
            if platform in ['iTunes', 'VUDU', 'Google Play', 'Amazon Prime Video', 'AppleTVApp']:
                master = pd.DataFrame(r).T  
                platform_found = True
                break
            if pd.isnull(r['platform']) or r['platform'] == '':
                continue

    if not platform_found:
        print("No valid platform found in res2")

    batch_size = 50000

    for batch_start in range(0, len(res2), batch_size):
        batch_end = min(batch_start + batch_size, len(res2))
        batch_res2 = res2[batch_start:batch_end]

        for r in batch_res2:
            count = 0
            mapped = 0

            for _, row1 in r.iterrows():
                if pd.isnull(row1['episode_name']) or row1['episode_name'] == '':
                    continue

                for _, row2 in master.iterrows():
                    matching_points = {
                        'id': id_,
                        'vonly_asset_id_dest': row1['vonly_asset_id'],
                        'vonly_asset_id_source': row2['vonly_asset_id'],
                        'season_no_dest': row1['season_no'],
                        'season_no_source': row2['season_no'],
                        'season_name_dest': row1['season_name'],
                        'season_name_source': row2['season_name'],
                        'episode_platform_dest': row1['platform'],
                        'episode_platform_source': row2['platform'],
                        'episode_portal_item_id_dest': row1['episode_portal_item_id'],
                        'episode_portal_item_id_source': row2['episode_portal_item_id'],
                        'episode_no_dest': row1['episode_no'],
                        'episode_no_source': row2['episode_no'],
                        'episode_names': 0,
                        'episode_name_dest': row1['episode_name'],
                        'episode_name_source': row2['episode_name'],
                        'count': 0,
                        'mapped': 0,
                    }

                    if row1['episode_no'] == row2['episode_no']:
                        id_ += 1
                        
                        matching_points['id'] = id_
                        
                        matching_points['episode_no_dest'] = row1['episode_no']
                        matching_points['episode_no_source'] = row2['episode_no']


                        if not pd.isnull(row2['episode_name']):
                            v = row2['episode_name'].lower()
                            plt2 = v.find("pilot")
                            u = row1['episode_name'].lower()
                            plt = u.find("pilot")

                            if plt2 == -1 and plt == -1:
                                m = token(row1['episode_name'])
                                n = token(row2['episode_name'])
                                if m >= n and m.find(n) != -1:
                                    count += 1
                                elif n > m and n.find(m) != -1:
                                    count += 1
                                    
                        
                        matching_points['episode_name_dest'] = row1['episode_name']
                        matching_points['episode_name_source'] = row2['episode_name']
                        
                        matching_points['vonly_asset_id_dest'] = row1['vonly_asset_id']
                        matching_points['vonly_asset_id_source'] = row2['vonly_asset_id']
                        
                        matching_points['season_no_dest'] = row1['season_no']
                        matching_points['season_no_source'] = row2['season_no']
                        
                        matching_points['season_name_dest'] = row1['season_name']
                        matching_points['season_name_source'] = row2['season_name']
                        
                        matching_points['episode_platform_dest'] = row1['platform']
                        matching_points['episode_platform_source'] = row2['platform']
                        
                        matching_points['episode_portal_item_id_dest'] = row1['episode_portal_item_id']
                        matching_points['episode_portal_item_id_source'] = row2['episode_portal_item_id']
                        
                        matching_points['episode_names'] = count

                        mapped = 1 if count >= 5 else 0

                        matching_points['count'] = count
                        matching_points['mapped'] = mapped


                        mapping_list.append(matching_points)


    return mapping_list


In [25]:
vds = get_v_id()

In [26]:
v_id = vds['vonly_asset_id']

In [27]:
len(v_id)

204998

In [28]:
ans=[]
ans_1 = []
for i in v_id:
    df0 = get_data(i)

    res = get_season_metadata(df0)

    res2 = get_episode_details(df0)
    res_2 = []
    for y in res2:
        m = y.drop_duplicates(subset = ['episode_name'])
        res_2.append(m)

    u = episode_validation1(res_2)
    ans_1 += u
    ans = ans + (metadata_validation(res))

No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid platform found in res2
No valid

DatabaseError: Execution failed on sql: 
        SELECT ti.vonly_asset_id,ti.title season_name,ti.portal_item_id season_portal_item_id,ti.num season_no,
                                                t.description,t.distributed_by,t.distributed_by_parent,t.produced_by,t.produced_by_parent,
                                                t.us_theater_release_date,t.director,t.actors,t.writers,t.producers,
                                                t.runtime,t.num season_no2,tie.title episode_name,tie.portal_item_id episode_portal_item_id, tie.num episode_no, ti.platform, 
                                                (select count(1) from tv_ids tii where tii.scope='tv_season' and tii.vonly_asset_id=ti.vonly_asset_id and tii.platform=ti.platform) va_count
                                                FROM vonly_data_feed_us_staging.tv_ids ti
                                                INNER JOIN sandbox.tvshows t ON ti.id=t.vonly_id AND ti.region=t.region 
                

In [21]:
make_excel(ans)
make_excel(ans_1)